In [1]:
import os
import numpy as np
import random
from scipy import ndimage
from skimage import transform
import keras
from keras.layers import Dense
from keras.applications import VGG16
from keras.models import Model
from keras.optimizers import SGD
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import skimage.io as io

# MEAN = np.array([123.0, 117.0, 104.0])
MEAN = np.array([0, 0, 0])
def getModel( output_dim ):
    ''' 
        * output_dim: the number of classes (int)
        
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-2].output #Last FC layer's output  
    softmax_layer = Dense(256, activation='softmax')(vgg_out) #Create softmax layer taking input as vgg_ou
    #Create new transfer learning model
    tl_model = Model( input=vgg_model.input, output=softmax_layer )

    #Freeze all layers of VGG16 and Compile the model
    #Confirm the model is appropriate
    for l in vgg_model.layers:
        l.trainable = False

    return tl_model
def convert_image(collect):
#     sz = np.shape(collect)[0]
    sz = len(collect)
    images = np.zeros((sz, 224, 224, 3))
    for i in range(sz):
        img = collect[i]
        img = transform.resize(img, (224, 224))*255
        if (len(img.shape) != 3):
           img = img[:,:,np.newaxis]
        images[i] = img
    return images

sizeOfEachClass = 4
sizeOfTestEachClass = 16
path='/home/yiluo/Homework/253/3/256_ObjectCategories/'
files = os.listdir(path)
X_train, X_val, X_test = [], [], []
Y_train, Y_val, Y_test = [], [], []
label = 0
for category in files:
    sub_path = path + category
    sub_files = os.listdir(sub_path)
    cnt = np.shape(sub_files)[0]
    num = 1
    for img_nm in sub_files:
        if img_nm[-4:] == '.jpg':
            img_path = sub_path + '/' + img_nm
            #img = ndimage.imread(sub_path + '/' + img_nm)
            #img = transform.resize(img, (224, 224))*255
            if num <= np.floor(0.8*cnt):
                if num <= sizeOfEachClass:
                    X_train.append(img_path)
                    Y_train.append(label)
            elif num <= np.floor(0.9*cnt):
                if num <= np.floor(0.8*cnt) + sizeOfTestEachClass:
                    X_val.append(img_path)
                    Y_val.append(label)
            else:
                X_test.append(img_path)
                Y_test.append(label)
            num += 1
    label += 1

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN not available)


In [3]:
#shuffle
ind=range(int(len(X_train)))
random.shuffle(ind)
X_train = [X_train[i] for i in ind]
Y_train = [Y_train[i] for i in ind]
Y_train_label = np.zeros((len(Y_train), 256))
for i in range(len(Y_train)):
    Y_train_label[i, Y_train[i]] = 1
    
#validation  
Y_val_label = np.zeros((len(Y_val), 256))
for i in range(len(Y_val)):
    Y_val_label[i, Y_val[i]] = 1
    
#test
Y_test_label = np.zeros((len(Y_test), 256))
for i in range(len(Y_test)):
    Y_test_label[i, Y_test[i]] = 1

In [6]:
#Output dim for your dataset
output_dim = 256 #For Caltech256

#Data
coll = io.ImageCollection(X_train)
X_train_data = convert_image(coll)
# coll_val = io.ImageCollection(X_val)
# X_val_data = convert_image(coll_val)

In [7]:
#Model
tl_model = getModel( output_dim )
# rmsprop = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0) # error
# sgd = keras.optimizers.SGD(lr=0.0001, momentum=0.9, decay=0.0, nesterov=True)
tl_model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [8]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    samplewise_center=True)

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
datagen.fit(X_train_data)

In [9]:
hist = tl_model.fit(X_train_data, Y_train_label, 
             batch_size=32, nb_epoch=20, 
             verbose=1, validation_split=0.5,
             shuffle=None, class_weight=None, sample_weight=None)

Train on 512 samples, validate on 512 samples
Epoch 1/20
512/512 [==============================] - 20s - loss: 7.0990 - acc: 0.0234 - val_loss: 6.4300 - val_acc: 0.0469
Epoch 2/20
512/512 [==============================] - 21s - loss: 2.1149 - acc: 0.4902 - val_loss: 5.6603 - val_acc: 0.1211
Epoch 3/20
512/512 [==============================] - 18s - loss: 0.5266 - acc: 0.9160 - val_loss: 5.4473 - val_acc: 0.1543
Epoch 4/20
512/512 [==============================] - 10s - loss: 0.1737 - acc: 0.9961 - val_loss: 5.4175 - val_acc: 0.1660
Epoch 5/20
512/512 [==============================] - 10s - loss: 0.0847 - acc: 1.0000 - val_loss: 5.4118 - val_acc: 0.1699
Epoch 6/20
512/512 [==============================] - 10s - loss: 0.0531 - acc: 1.0000 - val_loss: 5.4059 - val_acc: 0.1738
Epoch 7/20
512/512 [==============================] - 10s - loss: 0.0368 - acc: 1.0000 - val_loss: 5.3969 - val_acc: 0.1836
Epoch 8/20
512/512 [==============================] - 10s - loss: 0.0265 - acc: 1.0000

In [10]:
print(hist.history)

{'acc': [0.0234375, 0.490234375, 0.916015625, 0.99609375, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 'loss': [7.0989522337913513, 2.1149047836661339, 0.52656716853380203, 0.17368504870682955, 0.084748305380344391, 0.053056184202432632, 0.036764368182048202, 0.026489594718441367, 0.019367762259207666, 0.01424438483081758, 0.010526108206249774, 0.0078323656925931573, 0.0058857044496107846, 0.0044771934481104836, 0.0034520718472776935, 0.0026990597980329767, 0.0021403353021014482, 0.0017221575471921824, 0.001407113712048158, 0.0011684711571433581], 'val_acc': [0.046875, 0.12109375, 0.154296875, 0.166015625, 0.169921875, 0.173828125, 0.18359375, 0.1875, 0.1953125, 0.19921875, 0.201171875, 0.201171875, 0.205078125, 0.208984375, 0.216796875, 0.216796875, 0.224609375, 0.2265625, 0.2265625, 0.2265625], 'val_loss': [6.429975301027298, 5.6602542400360107, 5.4473003447055817, 5.4174874722957611, 5.4117568731307983, 5.4058922231197357, 5.3968522250652313, 5.38